In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt
import os
from tensorflow import keras
from PIL import Image

2024-08-31 17:38:22.292391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-31 17:38:22.303075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 17:38:22.313663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 17:38:22.317184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-31 17:38:22.326779: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import cv2

In [3]:
master_dir = "v_2"
maps = ["urban","agri","barrenland","grassland"]
for i in os.listdir(master_dir):
    print(i)

agri
barrenland
grassland
urban


In [6]:
x = []
y = []
img_size = 128
for i in os.listdir(master_dir):
    if i==".DS_Store":
        continue
    subfolder = master_dir+"/"+i
    input_folder = subfolder+"/"+"s1"
    output_folder = subfolder+"/"+'s2'
    print(input_folder)
    print(output_folder)
    for file in os.listdir(output_folder)[:1000]:
        if file.endswith(('.png','.jpeg','.jpg')):
            img_path = os.path.join(output_folder, file)
            img = Image.open(img_path).resize( ( img_size , img_size ))
            img_array = np.array(img, dtype=np.float64)
            img_array/=255
            y.append(img_array)
    for file in os.listdir(input_folder)[:1000]:
        if file.endswith(('.png','.jpeg','.jpg')):
            img_path = os.path.join(input_folder, file)
            img = Image.open(img_path).convert('L').resize( ( img_size , img_size ))
            img_array = np.array(img, dtype=np.float64)
            img_array = img_array.reshape((img_size, img_size, 1))
            img_array/=255
            x.append(img_array)

v_2/agri/s1
v_2/agri/s2
v_2/barrenland/s1
v_2/barrenland/s2
v_2/grassland/s1
v_2/grassland/s2
v_2/urban/s1
v_2/urban/s2


In [7]:
x = np.array(x)
y = np.array(y)

In [7]:
x.shape, y.shape

((400, 128, 128, 1), (400, 128, 128, 3))

In [8]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=56, shuffle=True)

In [9]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((340, 128, 128, 1), (340, 128, 128, 3), (60, 128, 128, 1), (60, 128, 128, 3))

In [10]:
from keras.layers import Conv2D, LeakyReLU, Concatenate, Conv2DTranspose, Input
from keras.models import Model
from keras.optimizers import Adam

img_size = 128

In [11]:
def get_generator_model():
    inputs = Input(shape=(img_size, img_size, 1))

    # Reduced number of filters and layers
    conv1 = Conv2D(8, kernel_size=(3, 3), strides=1)(inputs)
    conv1 = LeakyReLU()(conv1)
    conv1 = Conv2D(16, kernel_size=(3, 3), strides=1)(conv1)
    conv1 = LeakyReLU()(conv1)

    bottleneck = Conv2D(16, kernel_size=(3, 3), strides=1, activation='tanh', padding='same')(conv1)

    concat_1 = Concatenate()([bottleneck, conv1])
    conv_up_1 = Conv2DTranspose(16, kernel_size=(3, 3), strides=1, activation='relu')(concat_1)
    conv_up_1 = Conv2DTranspose(8, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_1)
    conv_up_1 = Conv2DTranspose(3, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_1)

    model = Model(inputs, conv_up_1)
    return model

In [12]:
generator = get_generator_model()
generator.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error')

generator.summary()

I0000 00:00:1725125290.813910   12079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725125290.850926   12079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725125290.850995   12079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725125290.858556   12079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725125290.858713   12079 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 126, 126,  │         80 │ input_layer[0][0] │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 126, 126,  │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 124, 124,  │      1,168 │ leaky_re_lu[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 124, 124,  │          0 │ conv2d_1[0][0]    │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 124, 124,  │      2,320 │ leaky_re_lu_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 124, 124,  │          0 │ conv2d_2[0][0],   │
│ (Concatenate)       │ 32)               │            │ leaky_re_lu_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 126, 126,  │      4,624 │ concatenate[0][0] │
│ (Conv2DTranspose)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 128, 128,  │      1,160 │ conv2d_transpose… │
│ (Conv2DTranspose)   │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, 130, 130,  │        219 │ conv2d_transpose… │
│ (Conv2DTranspose)   │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,571 (37.39 KB)

 Trainable params: 9,571 (37.39 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from keras.layers import MaxPooling2D, Flatten, Dense
from keras.models import Sequential

def create_discriminator():
    model = Sequential([
        # Reduced number of Conv2D layers and filters
        Conv2D(16, kernel_size=(3,3), strides=1, input_shape=(128, 128, 3), activation='relu'),
        MaxPooling2D(),
        Conv2D(32, kernel_size=(3,3), strides=1, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(32, activation='relu'),  # Reduced neurons
        Dense(1, activation='sigmoid')
    ])
    return model

discriminator = create_discriminator()
discriminator.compile(optimizer=Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.summary()

/home/omen/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 28800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       921,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 926,753 (3.54 MB)

 Trainable params: 926,753 (3.54 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
discriminator.compile(optimizer=Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
from keras.losses import BinaryCrossentropy, MeanSquaredError
cross_entropy = BinaryCrossentropy()
mse = MeanSquaredError()

def discriminator_loss(real_output, fake_output):
    real_labels = tf.ones_like(real_output) - 0.1 * tf.random.uniform(tf.shape(real_output))
    fake_labels = 0.1 * tf.random.uniform(tf.shape(fake_output))
    real_loss = cross_entropy(real_labels, real_output)
    fake_loss = cross_entropy(fake_labels, fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output, real_y):
    loss = mse(fake_output, tf.cast(real_y, tf.float32))
    return loss

In [17]:
generator_optimizer = Adam(0.0005)
discriminator_optimizer = Adam(0.0005)

@tf.function
def train_step(x_input, y_actual):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        y_gen = generator(x_input, training=True)
        real_output = discriminator(y_actual, training=True)
        generated_output = discriminator(y_gen, training=True)
        gen_loss = generator_loss(y_gen, y_actual)
        disc_loss = discriminator_loss(real_output, generated_output)
    
    generator_gradient = gen_tape.gradient(gen_loss, generator.trainable_variables)
    discriminator_gradient = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(generator_gradient, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradient, discriminator.trainable_variables))
    return gen_loss, disc_loss

epochs = 1000
batch_size = 32  # Reduced batch size
num_batches = x_train.shape[0] // batch_size

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch in range(num_batches):
        start = batch * batch_size
        end = (batch + 1) * batch_size
        x_batch = x_train[start:end]
        y_batch = y_train[start:end]
        gen_loss, disc_loss = train_step(x_batch, y_batch)
        print(f"Batch {batch+1}/{num_batches} - Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()}")
    
    if (epoch + 1) % 10 == 0:
        print(f"Saving weights at epoch {epoch + 1}")
        generator.save_weights('generator.weights.h5')
        discriminator.save_weights('discriminator.weights.h5')

Epoch 1/1000


ValueError: in user code:

    File "/tmp/ipykernel_12079/2427402889.py", line 9, in train_step  *
        generated_output = discriminator(y_gen, training=True)
    File "/home/omen/anaconda3/lib/python3.12/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/omen/anaconda3/lib/python3.12/site-packages/keras/src/layers/input_spec.py", line 227, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling Sequential.call().
    
    [1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 28800, but received input with shape (32, 30752)[0m
    
    Arguments received by Sequential.call():
      • inputs=tf.Tensor(shape=(32, 130, 130, 3), dtype=float32)
      • training=True
      • mask=None
